In [1]:
cd ..

/home/hantoine/Documents/Cours/Concordia/2019-Winter/SOEN_691_Big_Data_Analytics/project/accident-prediction-montreal


In [2]:
%matplotlib inline
import sys
sys.path.insert(0, '/home/hantoine/concordia/bigdata/project/spark/python')

In [3]:
from preprocess import get_negative_samples, get_positive_samples
from utils import init_spark
spark = init_spark()
spark.version

Spark Session created
Parameters:
	spark.driver.extraClassPath: ./data/xgboost4j-spark-0.72.jar:./data/xgboost4j-0.72.jar
	spark.network.timeout: 300s
	spark.app.id: local-1555946583229
	spark.master: local[10]
	spark.executor.id: driver
	spark.driver.host: laptop-hantoine.wireless.concordia.ca
	spark.app.name: Accident prediction
	spark.cleaner.periodicGC.interval: 5min
	spark.serializer: org.apache.spark.serializer.KryoSerializer
	spark.driver.memory: 7g
	spark.driver.port: 35023
	spark.rdd.compress: True
	spark.serializer.objectStreamReset: 100
	spark.submit.pyFiles: 
	spark.submit.deployMode: client
	spark.ui.showConsoleProgress: true


'3.0.0-SNAPSHOT'

In [4]:
# from weather import get_weather_df
# from pyspark.sql.functions import col
# weather_df = get_weather_df(spark, None)
# (weather_df
#  .filter((col('date') == datetime.fromisoformat('2014-01-17').date()) & (col('hour') == 15))
#  .select('station_id', 'rel_hum', 'visibility', 'stn_press', 'temp', 'risky_weather')
#  .dropna(how='all', subset=['rel_hum', 'visibility', 'stn_press', 'temp'])
#  .show())

# from preprocess import get_weather_information
# from weather import get_weather_df

# weather_df = get_weather_df(spark, None)
# (get_weather_information(sample, weather_df)
#  .select('rel_hum', 'visibility', 'stn_press', 'temp', 'risky_weather')
#  .show())

In [5]:
# from datetime import datetime
# sample = spark.createDataFrame([(0, datetime.fromisoformat('2014-01-17').date(), 15, -73.66106428971901, 45.48674117457887)],
#                       ['sample_id', 'date', 'hour', 'loc_long', 'loc_lat'])
# sample.show()

In [6]:
neg_samples = get_negative_samples(spark).sample(0.5)
pos_samples = get_positive_samples(spark)

In [7]:
# from pyspark.sql.functions import concat, lit, col
# weather_sum_pos = (pos_samples
#                    .select('visibility', 'stn_press', 'hmdx', 'rel_hum','dew_point_temp',
#                            'risky_weather', 'temp', 'wind_spd', 'wind_dir')
#                    .summary()
#                    .withColumn('v', concat(col('summary'), lit(' of positives'))))
# weather_sum_neg = (neg_samples
#                    .select('visibility', 'stn_press', 'hmdx', 'rel_hum','dew_point_temp',
#                            'risky_weather', 'temp', 'wind_spd', 'wind_dir')
#                    .summary()
#                    .withColumn('v', concat(col('summary'), lit(' of negatives'))))
# weather_sum = weather_sum_pos.union(weather_sum_neg)#.orderBy('summary').drop('summary')
# weather_sum.toPandas().set_index('v')

In [8]:
# from accidents_montreal import get_accident_df
# from weather import get_weather_station_coords_df
# from operator import and_
# acc = get_accident_df(spark)
# weather_stations = get_weather_station_coords_df(spark, None)
# def write_to_KML_file(row):
#     with open(f'./data/kml_stations/{str(row.station_id)}.kml', 'w') as file:
#         file.write(f"""<?xml version="1.0" encoding="UTF-8"?>
# <kml xmlns="http://www.opengis.net/kml/2.2">
#   <Document>
#   <Placemark>
#     <name>Station {row.station_id}</name>
#     <Point>
#       <coordinates>{row.station_long},{row.station_lat},0</coordinates>
#     </Point>
#   </Placemark>
#   </Document>
# </kml>
# """)
#     return True
# weather_stations.rdd.map(write_to_KML_file).reduce(and_)

In [9]:
imbalance_ratio = (neg_samples.count()/pos_samples.count())
print(f"Imbalance ratio: {imbalance_ratio}")

Imbalance ratio: 8.62468305958108


In [10]:
from preprocess import get_dataset_df
train_set, test_set = get_dataset_df(spark, pos_samples, neg_samples)
train_set, test_set = train_set.persist(), test_set.persist()
train_set.count()

Skip extracting road features: already done


1000192

In [11]:
# from pyspark.sql.functions import col
# from datetime import datetime
# df_sample = df.filter(col('date') > datetime.fromisoformat('2017-01-01'))
# df_sample.count()

In [12]:
# from random_undersampler import RandomUnderSampler

# (train_set, test_set) = df.randomSplit([0.7, 0.3])
# ru = RandomUnderSampler().setIndexCol('id')
# model = ru.fit(train_set)
# train_set = model.transform(train_set)
# test_set = model.transform(test_set)
# print(train_set.count(), test_set.count())

In [13]:
# from pyspark.sql.functions import udf, col
# from pyspark.sql.types import ArrayType, DoubleType
# from random_forest import features_col
# dayofweek_features = [f'{features_col[-1]}_{i}' for i in range(1, 8)]
# feature_names = features_col[:-1] + dayofweek_features

# def extract_X(samples):
#     def to_array(col):
#         def to_array_(v):
#             return v.toArray().tolist()
#         return udf(to_array_, ArrayType(DoubleType()))(col)
#     X = (samples
#          .select(to_array('features').alias('fa'))
#          .select([col("fa")[i] for i in range(26)]))
#     X = X.toDF(*feature_names)
#     #return X
#     return X.toPandas()#.values

# trainX = extract_X(train_set)
# # testX = extract_X(test_set)
# trainy = train_set.select('label').toPandas().values.T
# # trainx = test_set.select('label').toPandas().values.T

In [14]:
# trainX.count()

In [15]:
# from sklearn.metrics import precision_recall_curve
# from sklearn.metrics import auc
# from sklearn.ensemble import RandomForestClassifier
# from matplotlib import pyplot

# model = RandomForestClassifier(n_estimators=100)
# model.fit(trainX, trainy)
# # predict probabilities
# probs = model.predict_proba(testX)
# # keep probabilities for the positive outcome only
# probs = probs[:, 1]
# # predict class values
# yhat = model.predict(testX)
# # calculate precision-recall curve
# precision, recall, thresholds = precision_recall_curve(testy, probs)
# # calculate precision-recall AUC
# auc = auc(recall, precision)
# print('auc=%.3f' % auc)
# # plot no skill
# pyplot.plot([0, 1], [0.5, 0.5], linestyle='--')
# # plot the precision-recall curve for the model
# pyplot.plot(recall, precision, marker='.')
# # show the plot
# pyplot.show()

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator
from pyspark.ml import Pipeline
from random_undersampler import RandomUnderSampler
from class_weighter import ClassWeighter



# (train_set, test_set) = df.randomSplit([0.8, 0.2])

rf = RandomForestClassifier(labelCol="label",
                            featuresCol="features",
                            cacheNodeIds=True,
                            maxDepth=17,
                            impurity='entropy',
                            featureSubsetStrategy='sqrt',
                             weightCol='weight',
                            minInstancesPerNode=10,
                            numTrees=100,
                            subsamplingRate=1.0,
                            maxMemoryInMB=768
                           )
ru = RandomUnderSampler().setIndexCol('id').setTargetImbalanceRatio(1.0)
cw = ClassWeighter().setClassWeight([1/imbalance_ratio, 1.0])
pipeline = Pipeline().setStages([
#                                  ru,
                                 cw,
                                 rf])
model = pipeline.fit(train_set)

In [ ]:
# paramGrid = (ParamGridBuilder()
#      .addGrid(rf.numTrees, [100])
#     .addGrid(rf.featureSubsetStrategy, ['sqrt'])
#     .addGrid(rf.impurity, ['entropy'])
#     .addGrid(rf.maxDepth, [17])
#      .addGrid(rf.minInstancesPerNode, [10])
#      .addGrid(rf.subsamplingRate, [1.0])
# #     .addGrid(ru.targetImbalanceRatio, [1.0, 2.0, 3.0, 5.0, 7.0])
#     .addGrid(ru.targetImbalanceRatio, [1.0])
# #     .addGrid(cw.classWeight, [[1/imbalance_ratio, 1.0]])
#     .build())
# PR_evaluator = BinaryClassificationEvaluator(labelCol="label",
#                                              rawPredictionCol="rawPrediction",
#                                              metricName="areaUnderPR")
# tvs = CrossValidator(estimator=pipeline,
#                            estimatorParamMaps=paramGrid,
#                            evaluator=PR_evaluator,
# #                            trainRatio=0.7,
#                            collectSubModels=True)
# model = tvs.fit(train_set)
# model.avgMetrics

In [ ]:
def print_params(model):
    for stage in model.stages:
        params = stage.extractParamMap()
        for k in params:
            print(f'{k.name}: {params[k]}')

print_params(model)

In [ ]:
# predictions = model.transform(test_set)

In [ ]:
# Since test_set does not contain ids and random undersmapler is expecting one
# and we don't want to add id to test_set since some of them could match the ones in the training set 
# which would result in test examples to be removed
# And Since random sampler should be identiy for the test set, let juste call transform of the rf stage
predictions = model.stages[1].transform(test_set).persist()

start from here

In [ ]:
from evaluate import evaluate_binary_classifier
evaluate_binary_classifier(predictions)

In [ ]:
ROC_evaluator = \
    BinaryClassificationEvaluator(labelCol="label",
                                  rawPredictionCol="rawPrediction",
                                  metricName="areaUnderROC")
area_under_ROC = ROC_evaluator.evaluate(predictions)
area_under_ROC

In [ ]:
train_predictions = model.transform(train_set).persist()

In [ ]:
from evaluate import evaluate_binary_classifier
evaluate_binary_classifier(train_predictions)

In [ ]:
ROC_evaluator = \
    BinaryClassificationEvaluator(labelCol="label",
                                  rawPredictionCol="rawPrediction",
                                  metricName="areaUnderROC")
area_under_ROC = ROC_evaluator.evaluate(train_predictions)
area_under_ROC

In [ ]:
from random_forest import features_col
import pandas as pd
fi = pd.DataFrame(model
                  .stages[1]
                  .featureImportances
                  .toArray())
fi.index = features_col
fi.columns = ['Feature Importances']
fi = fi.sort_values(by=['Feature Importances'], ascending=False)
fi.plot.bar()

In [ ]:
from random_forest import compute_precision_recall_graph
precision_recall = compute_precision_recall_graph(predictions, 20)

In [ ]:
precision_recall.plot(x='Threshold', title='RF')

In [ ]:
# precision_recall.plot(x='Recall', y='Precision', ylim=(0,1))

In [ ]:
precision_recall

In [ ]:
# precision_recall.to_csv('results/rf_pr_tab.csv')